In [11]:
import requests
import pandas as pd
from clarifai import rest
from clarifai.rest import ClarifaiApp
import time
import os

In [72]:
def clarifaize(img):
    app = ClarifaiApp("mrKZLVJBD6mWwmzTg_84YsbDlZRj1yQPIlBlfQZd", "vuuKXEtODxS8RHNo-a0wfF4vMJr4pVojR-mEScL3")
    # get the general model
    model = app.models.get("general-v1.3")
    # predict with the model
#     prediction = model.predict_by_url(url=img)['outputs'][0]['data']['concepts']
    prediction = model.predict_by_filename(img)['outputs'][0]['data']['concepts']
    tags = pd.DataFrame({'Tag': [], 'Probability': []})
    for item in prediction:
        tags = tags.append(pd.DataFrame({'Tag': [item['name']], 'Probability': [item['value']]}))
#     return tags.reset_index(drop=True)
    tags = list(tagDataset['Tag'])
    if('no person' in tags):
        tags.remove('no person')
    
    return tags

In [73]:
# tagDataset = clarifaize('http://miriadna.com/desctopwalls/images/max/Forest-small-river.jpg')
tagDataset = clarifaize('nasa.jpg')

In [69]:
# tags = tagDataset['Tag']
# tagPhrase = " ".join(tagDataset['Tag'])
tags = list(tagDataset['Tag'])
if ('no person' in list(tagDataset['Tag'])):
    tags.remove('no person')
    tagPhrase = " ".join(tags)
    


In [82]:
tagPhrase = " ".join(tagDataset)

In [83]:
tagPhrase

u'astronomy moon ball-shaped space galaxy infinity planet nebula exploration science fantasy astrology surreal plasma cosmos solar system stellar imagery solar orbit'

In [75]:
urls = []
for tag in tags:
    urls.append("https://favqs.com/api/quotes/?filter=" + tag)
#     print(tag)

In [76]:
urls

[u'https://favqs.com/api/quotes/?filter=astronomy',
 u'https://favqs.com/api/quotes/?filter=moon',
 u'https://favqs.com/api/quotes/?filter=ball-shaped',
 u'https://favqs.com/api/quotes/?filter=space',
 u'https://favqs.com/api/quotes/?filter=galaxy',
 u'https://favqs.com/api/quotes/?filter=infinity',
 u'https://favqs.com/api/quotes/?filter=planet',
 u'https://favqs.com/api/quotes/?filter=nebula',
 u'https://favqs.com/api/quotes/?filter=exploration',
 u'https://favqs.com/api/quotes/?filter=science',
 u'https://favqs.com/api/quotes/?filter=fantasy',
 u'https://favqs.com/api/quotes/?filter=astrology',
 u'https://favqs.com/api/quotes/?filter=surreal',
 u'https://favqs.com/api/quotes/?filter=plasma',
 u'https://favqs.com/api/quotes/?filter=cosmos',
 u'https://favqs.com/api/quotes/?filter=solar system',
 u'https://favqs.com/api/quotes/?filter=stellar',
 u'https://favqs.com/api/quotes/?filter=imagery',
 u'https://favqs.com/api/quotes/?filter=solar',
 u'https://favqs.com/api/quotes/?filter=orbi

In [77]:
headers = {'Authorization' : 'Token token="b09904a4d885299ee386f5cbbf795c56"'}
req = requests.get("https://favqs.com/api/quotes/?filter=mountain", headers = headers)
def get_quotes(urls):
    quotes = pd.DataFrame({'Quote': [], 'Author': []})
    for url in urls:
        req = requests.get(url, headers = headers).json()
        time.sleep(0.5)
        print "Processing: " + url
        for item in req['quotes']:
            if item['body'] != 'No quotes found':
                quotes = quotes.append(pd.DataFrame({'Quote': [item['body']], 'Author': [item['author']]}))
    return quotes.reset_index(drop=True)

In [106]:
quotes = pd.DataFrame({'Quote': get_quotes(urls)['Quote'], 'Author': get_quotes(urls)['Author']})

Processing: https://favqs.com/api/quotes/?filter=astronomy
Processing: https://favqs.com/api/quotes/?filter=moon
Processing: https://favqs.com/api/quotes/?filter=ball-shaped
Processing: https://favqs.com/api/quotes/?filter=space
Processing: https://favqs.com/api/quotes/?filter=galaxy
Processing: https://favqs.com/api/quotes/?filter=infinity
Processing: https://favqs.com/api/quotes/?filter=planet
Processing: https://favqs.com/api/quotes/?filter=nebula
Processing: https://favqs.com/api/quotes/?filter=exploration
Processing: https://favqs.com/api/quotes/?filter=science
Processing: https://favqs.com/api/quotes/?filter=fantasy
Processing: https://favqs.com/api/quotes/?filter=astrology
Processing: https://favqs.com/api/quotes/?filter=surreal
Processing: https://favqs.com/api/quotes/?filter=plasma
Processing: https://favqs.com/api/quotes/?filter=cosmos
Processing: https://favqs.com/api/quotes/?filter=solar system
Processing: https://favqs.com/api/quotes/?filter=stellar
Processing: https://fav

In [107]:
# type(quotes)
quotes.tail()

,Author,Quote
225,Mitt Romney,"Even though I love solar and love wind, like m..."
226,Barack Obama,"Here at this site, Solyndra expects to make en..."
227,Greg Bear,Of all the planets apart from Earth in our sol...
228,Joseph Addison,No oppression is so heavy or lasting as that w...
229,Yahoo Serious,"Then if your movie clicks with real audiences,..."


In [108]:
type(tagPhrase)

unicode

In [109]:
phrases = quotes.append(pd.DataFrame({'Quote' : [tagPhrase], 'Author': ['No One']})).reset_index(drop=True)

In [110]:
phrases.tail()

,Author,Quote
226,Barack Obama,"Here at this site, Solyndra expects to make en..."
227,Greg Bear,Of all the planets apart from Earth in our sol...
228,Joseph Addison,No oppression is so heavy or lasting as that w...
229,Yahoo Serious,"Then if your movie clicks with real audiences,..."
230,No One,astronomy moon ball-shaped space galaxy infini...


In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [112]:
def similarity_matrix(dataframe):
    tfidf = TfidfVectorizer(analyzer = 'word', ngram_range = (1,3), min_df = 0, stop_words = 'english')
    tfidf_matrix = tfidf.fit_transform(dataframe)
    cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
    return pd.DataFrame(cosine_similarities)

In [113]:
matrix = similarity_matrix(phrases['Quote'])

In [114]:
def best_similarities(dataframe):
    return dataframe[len(dataframe)-1].sort_values(ascending=False)[1:7].reset_index()

In [115]:
best_similarities(matrix)

,index,230
0,191,0.100308
1,10,0.041405
2,172,0.041405
3,7,0.030362
4,214,0.030346
5,6,0.030306


In [117]:
phrases['Quote'][191] + ' - ' + phrases['Author'][191]

u'The cosmos is all that is, or ever was, or ever will be. - Carl Sagan'

In [118]:
def candidate_indexes(dataframe):
	return dataframe['index']

def candidate_captions(dataframe, indexes):
	captions = []
	for index in indexes:
		captions.append(dataframe[index])
	return captions

In [121]:
candidate_captions(phrases['Quote'] + ' - ' + phrases['Author'], candidate_indexes(best_similarities(matrix)))

[u'The cosmos is all that is, or ever was, or ever will be. - Carl Sagan',
 u'Before a war military science seems a real science, like astronomy but after a war it seems more like astrology. - Rebecca West',
 u'Before a war military science seems a real science, like astronomy but after a war it seems more like astrology. - Rebecca West',
 u'So fantasy was fine early on, and when I discovered science fiction, I was very happy with it, because my first interest in science fiction came with an interest in astronomy. - Octavia Butler',
 u"It's really kind of cool to have solar panels on your roof. - Bill Gates",
 u"I love biomedical science, I love astronomy, and you can't really do much with those in a fantasy setting. - Elizabeth Moon"]

In [129]:
alltags = []
for tag in tags[:7]:
    alltags.append("#"+tag)

In [130]:
" ".join(alltags)

u'#astronomy #moon #ball-shaped #space #galaxy #infinity #planet'